# Análisis de Viajes — Zuber (Chicago)

## Introducción

Trabajas como **analista de datos** para **Zuber**, una nueva empresa de viajes compartidos en **Chicago**. El objetivo es identificar **patrones en los datos** para entender el comportamiento de los pasajeros y evaluar cómo factores externos, como el **clima**, influyen en los viajes.

---

## Base de Datos

La base de datos contiene información histórica de **viajes en taxi en Chicago** y está compuesta por las siguientes tablas:

### `neighborhoods`

Información de los barrios.

* `name`: nombre del barrio
* `neighborhood_id`: identificador del barrio

### `cabs`

Información de los vehículos.

* `cab_id`: identificador del taxi
* `vehicle_id`: ID técnico del vehículo
* `company_name`: empresa propietaria

### `trips`

Información de los viajes.

* `trip_id`: identificador del viaje
* `cab_id`: taxi que realiza el viaje
* `start_ts`: inicio del viaje (redondeado a la hora)
* `end_ts`: fin del viaje (redondeado a la hora)
* `duration_seconds`: duración del viaje en segundos
* `distance_miles`: distancia en millas
* `pickup_location_id`: barrio de origen
* `dropoff_location_id`: barrio de destino

### `weather_records`

Información meteorológica.

* `record_id`: identificador del registro
* `ts`: fecha y hora del registro (redondeado a la hora)
* `temperature`: temperatura registrada
* `description`: descripción del clima

---

## Consideraciones

No existe una relación directa entre `trips` y `weather_records`. Para combinarlas, se utilizará la **hora de inicio del viaje** (`trips.start_ts`) y la **hora del registro meteorológico** (`weather_records.ts`).

---

## Objetivos del Análisis

### Análisis Exploratorio (Ejercicios 1–3)

* Explorar las tablas y sus relaciones.
* Analizar patrones de viajes y empresas.
* Identificar tendencias relevantes.

### Prueba de Hipótesis (Ejercicios 4–6)

Evaluar la siguiente hipótesis:

> **La duración de los viajes desde el Loop hasta el Aeropuerto Internacional O'Hare cambia durante los sábados lluviosos.**

Se compararán las duraciones de los viajes bajo distintas condiciones climáticas para validar estadísticamente la hipótesis.


1. Imprime el campo company_name. Encuentra la cantidad de viajes en taxi para cada compañía de taxis para el 15 y 16 de noviembre de 2017, asigna al campo resultante el nombre trips_amount e imprímelo también. Ordena los resultados por el campo trips_amount en orden descendente.

```SQL
SELECT 
    cabs.company_name,
    COUNT(trips.trip_id) AS trips_amount
FROM trips
JOIN cabs ON trips.cab_id = cabs.cab_id
WHERE trips.start_ts BETWEEN '2017-11-15 00:00:00' AND '2017-11-16 23:59:59'
GROUP BY cabs.company_name
ORDER BY trips_amount DESC;
```

2. Encuentra la cantidad de viajes para cada empresa de taxis cuyo nombre contenga las palabras "Yellow" o "Blue" del 1 al 7 de noviembre de 2017. Nombra la variable resultante trips_amount. Agrupa los resultados por el campo company_name.

```SQL
SELECT 
    cabs.company_name,
    COUNT(trips.trip_id) AS trips_amount
FROM trips
JOIN cabs ON trips.cab_id = cabs.cab_id
WHERE (cabs.company_name LIKE '%Yellow%' OR cabs.company_name LIKE '%Blue%') 
    AND trips.start_ts BETWEEN '2017-11-01 00:00:00' AND '2017-11-07 23:59:59'
GROUP BY cabs.company_name;
```

3. Del 1 al 7 de noviembre de 2017, las empresas de taxis más populares fueron Flash Cab y Taxi Affiliation Services. Encuentra el número de viajes de estas dos empresas y asigna a la variable resultante el nombre trips_amount. Junta los viajes de todas las demás empresas en el grupo "Other". Agrupa los datos por nombres de empresas de taxis. Asigna el nombre company al campo con nombres de empresas de taxis. Ordena el resultado en orden descendente por trips_amount.

```SQL
SELECT 
    CASE 
        WHEN cabs.company_name = 'Flash Cab' THEN 'Flash Cab'
        WHEN cabs.company_name = 'Taxi Affiliation Services' THEN 'Taxi Affiliation Services'
        ELSE 'Other'
    END AS company,
    COUNT(trips.trip_id) AS trips_amount
FROM trips
JOIN cabs ON trips.cab_id = cabs.cab_id
WHERE trips.start_ts BETWEEN '2017-11-01 00:00:00' AND '2017-11-07 23:59:59'
GROUP BY company
ORDER BY trips_amount DESC;
```

4. Recupera los identificadores de los barrios de O'Hare y Loop de la tabla neighborhoods.

```SQL
SELECT 
    neighborhood_id, 
    name
FROM neighborhoods
WHERE name IN ('O''Hare', 'Loop');
```

5. Para cada hora recupera los registros de condiciones meteorológicas de la tabla weather_records. Usando el operador CASE, divide todas las horas en dos grupos: Bad si el campo description contiene las palabras rain o storm, y Good para los demás. Nombra el campo resultante weather_conditions. La tabla final debe incluir dos campos: fecha y hora (ts) y weather_conditions.

```SQL
SELECT 
    ts,
    CASE
        WHEN description ILIKE '%rain%' OR description ILIKE '%storm%' 
            THEN 'Bad'
        ELSE 'Good'
    END AS weather_conditions
FROM weather_records
ORDER BY ts;
```

6. Recupera de la tabla de trips todos los viajes que comenzaron en el Loop (pickup_location_id: 50) el sábado y terminaron en O'Hare (dropoff_location_id: 63). Obtén las condiciones climáticas para cada viaje. Utiliza el método que aplicaste en la tarea anterior. Recupera también la duración de cada viaje. Ignora los viajes para los que no hay datos disponibles sobre las condiciones climáticas.

Las columnas de la tabla deben estar en el siguiente orden:

* `start_ts`
* `weather_conditions`
* `duration_seconds`
* `Ordena por trip_id`.

```SQL
SELECT 
    trips.start_ts,
    CASE
        WHEN weather_records.description ILIKE '%rain%' OR weather_records.description ILIKE '%storm%' 
            THEN 'Bad'
        ELSE 'Good'
    END AS weather_conditions,
    trips.duration_seconds
FROM trips
JOIN weather_records
    ON trips.start_ts = weather_records.ts
WHERE trips.pickup_location_id = 50
  AND trips.dropoff_location_id = 63
  AND EXTRACT(DOW FROM trips.start_ts) = 6
  ORDER BY t.trip_id;
```

# Análisis exploratorio de datos (Python)

Además de los datos que recuperaste en las tareas anteriores, ahora tienes un segundo archivo. En total cuentas con dos CSV:

- **/datasets/project_sql_result_01.csv**  
  Contiene los siguientes datos:  
  - `company_name`: nombre de la empresa de taxis  
  - `trips_amount`: número de viajes de cada compañía de taxis el 15 y 16 de noviembre de 2017  

- **/datasets/project_sql_result_04.csv**  
  Contiene los siguientes datos:  
  - `dropoff_location_name`: barrios de Chicago donde finalizaron los viajes  
  - `average_trips`: promedio de viajes que terminaron en cada barrio en noviembre de 2017

### Tareas a realizar
1. Importar los archivos.

In [4]:
# Librerías principales para análisis de datos
import pandas as pd
import numpy as np

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías para estadística e hipótesis
from scipy import stats

# Importar datasets
df_companies = pd.read_csv('Jorge-GraVaz/Proyecto-Sprint-8/datasets/project_sql_result_01.csv')
df_neighborhoods = pd.read_csv('Jorge-GraVaz/Proyecto-Sprint-8/datasets/project_sql_result_04.csv')

# Vista rápida para confirmar que se cargaron correctamente
print("Empresas de taxis:")
print(df_companies.head(), "\n")

print("Barrios de destino:")
print(df_neighborhoods.head())


FileNotFoundError: [Errno 2] No such file or directory: 'Jorge-GraVaz/Proyecto-Sprint-8/datasets/project_sql_result_01.csv'